# gameplay

> Fill in a module description here

In [4]:
#| default_exp gameplay

In [1]:
#| hide
from nbdev.showdoc import *

In [2]:
%load_ext autoreload


In [7]:
%autoreload 2

In [3]:
#| export
import clue.model as model
import clue.ui as ui
from random import choice,shuffle,choices
import os

In [4]:
#| hide
stgame = model.createClue(4)
stmoves = []

In [5]:
#| export
def getPlay(pc,game, moves):
    
    

    ui.showBoard(pc,game,moves)
    playerRoom = model.roomOf(pc,game)
    passages = model.roomsAround(playerRoom)
    locs = [[3,x] for x in passages ]
    verb = choice(["totter","walk","stumble","staggar","rush","dash","race","sprint"])


    print(model.people[pc-1][2] + " will " + verb + " to:\n")
 
    l = ui.chooseKey(locs,pc,game,moves)
    l = passages[l-1]

    wKeys = [[1,x] for x in range(len(model.people))]
    p = ui.chooseKey(wKeys,pc,game,moves) - 1

    wKeys = [[2,x] for x in range(len(model.weapons))]
    w = ui.chooseKey(wKeys,pc,game,moves) - 1

    return p, w, l



In [6]:
#| hide
#stguess = getPlay(1,stgame,stmoves)
#print(ui.showChoice(stguess))
def seenCounts(pc,game, moves):
    knoweledge = getSeen(pc,game, moves)
    p = [0,0,0]
    for x in knoweledge:
        i = int(x/100) -1
        p[i] += 1

    return p

In [12]:
#| hide
#stguess = getPlay(2,stgame,stmoves)

In [7]:
#| export




def simpleGuess(pc,game, moves):


    playerRoom = model.roomOf(pc,game)
    exclude  = set(model.getSeen(pc,game, moves))
    
    pdeck = [model.keyCard(1,x) for x in range(len(model.people))]
    pdeck = [x for x in  pdeck if x not in exclude]
    p = choice(pdeck)%100
    
    pdeck = [model.keyCard(2,x) for x in range(len(model.weapons))]
    pdeck = [x for x in pdeck if x not in exclude]
    w = choice(pdeck)%100
    
    pdeck = [model.keyCard(3,x) for x in model.roomsAround(playerRoom)]
    rdeck = [x for x in  pdeck if x not in exclude]
    
    if len(rdeck) == 0:
        l = pdeck[0]%100
    else:
        l = choice(rdeck)%100
    
    return (p,w,l)


In [8]:
#| export

def itemsForType(t):
    if t == 1:
        return model.people
    if t == 2:
        return model.weapons
    return model.rooms
        
def itemGuess(t,moves):
    l = itemsForType(t)

    ret = [1 for x in range(len(l))]
    for x in moves:
        ret[x[t-1]] += 1
    #print(ret)
    return ret
    
def showCounts(t,moves):
    items = itemGuess(t,moves)
    base = itemsForType(t)
    #print(items,base)
    for i in range(len(items)):
        print(i,ui.cardFull(base[i]),items[i])


                              




In [9]:
#| export
def biasWalk(a,b,c):
    
    def bWalk(exclude,pc,game, moves):
        playerRoom = model.roomOf(pc,game) 
        dest = model.roomsAround(playerRoom)
        if model.keyCard(3,dest[a]) not in exclude:
            return dest[a]%100
        if model.keyCard(3,dest[b]) not in exclude:
            return dest[b]%100
        
        return dest[c]%100
    return bWalk
    
 

In [10]:
#| export
def rWalk(exclude,pc,game, moves):
    playerRoom = model.roomOf(pc,game)
    pdeck = [model.keyCard(3,x) for x in model.roomsAround(playerRoom)]
    rdeck = [x for x in  pdeck if x not in exclude]
    
    if len(rdeck) == 0:
        l = pdeck[0]%100
    else:
        l = choice(rdeck)%100
                              
    return l

In [11]:
#| export


def roomsAway(roomNum,roomSize):
    foo = [x for x in range(roomSize)]
    middle = int( roomSize / 2)
    if roomNum <middle:
        lm = foo[roomNum - int(middle):]
        lSide = lm + foo[:roomNum] 
        lSide.reverse()
        rSide = foo[roomNum + 1:roomNum + 1 + middle]
    elif roomNum > middle:
        lSide = foo[roomNum-middle:roomNum]
        lSide.reverse()
        rSide = foo[roomNum +1: ] + foo[:roomNum-middle]
    else:
        lSide = foo[:middle]
        lSide.reverse()
        rSide = foo[middle+1:]
    return lSide, rSide

def repelWalk(exclude,pc,game, moves):
    playerRoom = model.roomOf(pc,game)
    lRooms, rRooms = roomsAway(playerRoom,len(model.rooms))
    #print("staring at " + ui.cardFull(model.rooms[playerRoom]))
    #print(playerRoom,lRooms,rRooms)
    #print(exclude)
    def score(rooms):
        rooms.reverse()
        weights = []
        cards = []
        for i in range(len(rooms)):
            r = rooms[i]
            cards.append(str(model.keyCard(3,r)) + " " + model.rooms[r][0])
            
            if model.keyCard(3,r) in exclude:
                w = 2**i
                #print(i,w)
                weights.append(w)
            else:
                weights.append(0)
        #print(cards,weights)
        return sum(weights)


    lPull = score(lRooms)
    rPull = score(rRooms)
    #trace.append([playerRoom,lPull,rPull])
    #print(trace)
    #ui.showBoard(pc,game,moves)
    achoice = model.roomsAround(playerRoom)
    if lPull > rPull:
        #print("to" + ui.cardFull(model.rooms[achoice[2]]))
        return achoice[2]
    if lPull < rPull:
        
        #print("to" + ui.cardFull(model.rooms[achoice[0]]))
        return achoice[0]
    #print("same")    
    return rWalk(exclude,pc,game, moves)


In [12]:
#| export
def mimic(memory,walk):
    def betterGuess(pc,game, moves):

        exclude  = set(model.getSeen(pc,game, moves))

        if memory < 0:
            examine = []
        else:
            examine = moves[-memory:]


        keydeck = [model.keyCard(1,x) for x in range(len(model.people))]
        pdeck = [x for x in  keydeck if x not in exclude]
        pC = itemGuess(1,examine)
        pWeights = [pC[x%100] * pC[x%100] for x in  keydeck if x not in exclude]

        p = choices(pdeck,weights=pWeights)
        p = p[0]%100

        keydeck = [model.keyCard(2,x) for x in range(len(model.weapons))]
        pdeck = [x for x in keydeck if x not in exclude]
        pC = itemGuess(2,examine)
        pWeights = [pC[x%100] * pC[x%100] for x in  keydeck if x not in exclude]

        w = choices(pdeck,weights=pWeights)[0]%100
        
        l = walk(exclude,pc,game, moves)

        return (p,w,l)
    
    return betterGuess

In [13]:
#| export
def cheater(walk):
    def betterGuess(pc,game, moves):

 
        exclude  = set(model.handOf(pc,game) +[x[-1] for x in moves] )

       
        examine = []
   


        keydeck = [model.keyCard(1,x) for x in range(len(model.people))]
        pdeck = [x for x in  keydeck if x not in exclude]
        pC = itemGuess(1,examine)
        pWeights = [pC[x%100] * pC[x%100] for x in  keydeck if x not in exclude]

        p = choices(pdeck,weights=pWeights)
        p = p[0]%100

        keydeck = [model.keyCard(2,x) for x in range(len(model.weapons))]
        pdeck = [x for x in keydeck if x not in exclude]
        pC = itemGuess(2,examine)
        pWeights = [pC[x%100] * pC[x%100] for x in  keydeck if x not in exclude]

        w = choices(pdeck,weights=pWeights)[0]%100
        
        l = walk(exclude,pc,game, moves)

        return (p,w,l)
    return betterGuess


def halfRight(walk):
    oracle = cheater(walk)
    
    def betterGuess(pc,game, moves):
        p,w,l = oracle(pc,game, moves)
        exclude  = model.handOf(pc,game) +[x[-1] for x in moves] 
        exclude = [x for x in exclude if x < 300]
        

        if len(exclude) <= 0:
            return (p,w,l)

        wrong = choice(exclude)
        if wrong < 200:
            p = wrong%100
        else:
            w = wrong%100

        return (p,w,l)
    return betterGuess

In [15]:
#| export

def hasGuess(guess,moves):
    p, w, l = guess[:3]
    c = (p,w,l)
    for x in moves:
        p, w, l = x[:3]
        xc = (p,w,l)
        if c == xc:
            return True
    return False

def repeater(strats,verbose=False):
    def betterGuess(pc,game, moves):
        counter = 0
        for strat in strats:
            
            g =  strat(pc,game, moves)
            if not hasGuess(g,moves):
                if verbose and counter > 0:
                    print("counter " + str(counter))
                return g
            counter += 1
                        
        if verbose and counter > 0:
            print(" over counter " + str(counter))
        return strats[-1](pc,game, moves)
                          
    return betterGuess

In [16]:
r1 = repeater([mimic(1*(x+3),rWalk) for x in range(4)])
rgame = model.createClue(4)
r1(1,rgame,[])

(5, 3, 0)

## Visual

In [17]:
#| export
def playerName(i):
    return ui.cardFull(model.cardLookup(1,i-1))
    
def runVisualGame( strats):
    numPlayers = len(strats)
    done = False
    agame = model.createClue(numPlayers)
    amoves = []
    player = 1

   
             
    while not done:
        #print(seenCounts(player,agame,amoves))
        
        stratName, isUser, strat = strats[player - 1]
        
        if isUser:
            #ui.showMoves(player,amoves[-numPlayers:],numPlayers)
            ui.printPlayerHand(player,agame)
            ui.showMoves(player,amoves[-numPlayers:],numPlayers)

        #print(player,agame,amoves)
        ans = strat(player,agame,amoves)
        if isUser:
            pass
            #os.system("clear")
        agame = model.relocatePlayers(player , ans[0],ans[2],agame)
        #print( playerName(player) + " suspects " + ui.showChoice(ans)+".")
        check = model.getAnswer(player,agame,ans)
        
        if check[0] < 0:
            seen =  model.getSeen(player,agame, amoves)
            exclude  = set(seen)
            done = True
            for x in model.keysForGuess(ans):
                if x in exclude:
                    print("\n\n" + playerName(player) + " is not quite ready to accuse.")
                    done = False
            
        else:
            #(p,w,l,playerNum, c,y[0],y[1])
            card = check[-1]
            
            amoves.append(check)


        if not done:
            player = player + 1
            if player - 1 >=len(strats):
               player = 1 
    
    print("\n" + playerName(player) + " " + stratName + " won!")
    ui.printGame(agame) 
    ui.showMoves(player,amoves,numPlayers)

In [46]:
#| hide
trace = []
p1 = ("player",True,getPlay)
s1 = ("simple",False,repeater([mimic(-1,rWalk) for x in range(4)]),True)
b1 = ("better",False,repeater([mimic(-1,rWalk) for x in range(4)]),True)
#s2 = #repeater([("simpleR",False,mimic(-1,repelWalk)) for x in range(4)])
#s3 = #repeater([("stay",False,mimic(-1,biasWalk(1,0,2)))  for x in range(4)])
b1 = #repeater([("better",False,mimic(8,rWalk))  for x in range(4)])
#d1 = #repeater([("deep",False,mimic(20,rWalk))   for x in range(4)])
hr = ("halfRight",False,halfRight(rWalk))
strategies = [ s1,p1,s1,hr,hr] #hr,s1,p1,s1,s1]
runVisualGame(strategies)

ok
repeat
👠 Miss Scarlet, 🔫 Revolver, Billiard Room, Kitchen
👠  asks (🥼 &  🪠 in the Kitchen) ------->👠🧐🥼❓❓
╒═══════════╤══════════════════╤═════════════════╕
│ Ball Room │ 1) Billiard Room │ 2) Conservatory │
│    G      │                  │    S, M, W      │
├───────────┼──────────────────┼─────────────────┤
│ Study     │                  │ 3) Dining Room  │
├───────────┼──────────────────┼─────────────────┤
│ Library   │                  │ Hall            │
│    P      │                  │                 │
├───────────┼──────────────────┼─────────────────┤
│           │ Lounge           │ Kitchen         │
│           │    P             │                 │
╘═══════════╧══════════════════╧═════════════════╛
🧐 will race to:

1*Billiard Room
2 Conservatory                     👠
3 Dining Room


Choose from 1 to 3:  2


1*👠 Miss Scarlet
2 🧐 Colonel Mustard
3 🥼  Mrs. White
4 🐉 Mr. Green
5 🍆 Professor Plum
6 🦚 Mrs. Peacock


Choose from 1 to 6:  5


1 🔪 Knife
2*🔫 Revolver
3 🪢 Rope
4 🔧 Wrench
5 🕯️ Candlestick
6 🪠 Lead Pipe


...

Choose from 1 to 6:  4


1 🔪 Knife                          🥼
2*🔫 Revolver                       🐉, 🍆
3 🪢 Rope                           🐉, 🍆
4*🔧 Wrench
5 🕯️ Candlestick
6*🪠 Lead Pipe                      👠, 🧐, 🍆


Choose from 1 to 6:  5


ok
repeat
ok
repeat
repeat
👠 Miss Scarlet, 🔫 Revolver, Billiard Room, Kitchen
🧐  asks (🐉 &  🕯️ in the Hall) ---------->❓🧐🥼❓❓ shows Study
🥼  asks (👠 &  🕯️ in the Hall) ---------->👠🧐🥼🐉🍆
🐉  asks (🥼 &  🕯️ in the Hall) ---------->❓❓❓🐉🍆
🍆  asks (🦚 &  🪠 in the Kitchen) ------->👠🧐🥼❓🍆
👠  asks (🦚 &  🕯️ in the Hall) ---------->👠🧐❓❓❓
╒══════════════╤═══════════════╤══════════════╕
│ 3) Ball Room │ Billiard Room │ Conservatory │
├──────────────┼───────────────┼──────────────┤
│ 2) Study     │               │ Dining Room  │
│    M, P      │               │              │
├──────────────┼───────────────┼──────────────┤
│ 1) Library   │               │ Hall         │
│    W, G      │               │              │
├──────────────┼───────────────┼──────────────┤
│              │ Lounge        │ Kitchen      │
│              │               │    S, P      │
╘══════════════╧═══════════════╧══════════════╛
🧐 will race to:

1 Library                          🥼, 🐉, 🍆
2*Study                            👠, 🍆


KeyboardInterrupt: Interrupted by user

In [ ]:
#| export
def runGame( strats):
    numPlayers = len(strats)
    done = False
    agame = model.createClue(numPlayers)
    amoves = []
    player = 1
          
    while not done:
        #print(seenCounts(player,agame,amoves))
        
        stratName, isUser, strat = strats[player - 1]
        
        ans = strat(player,agame,amoves)
        agame = model.relocatePlayers(player , ans[0],ans[2],agame)
        check = model.getAnswer(player,agame,ans)
        
        if check[0] < 0:
            seen =  model.getSeen(player,agame, amoves)
            exclude  = set(seen)
            done = True
            for x in model.keysForGuess(ans):
                if x in exclude:
                    done = False
            
        else:
            #(p,w,l,playerNum, c,y[0],y[1])
            card = check[-1]
            
            amoves.append(check)

        if not done:
            player = player + 1
            if player - 1 >= numPlayers:
               player = 1 
    
    return player, agame , amoves


In [ ]:
#| hide
s1 = ("simple",False,mimic(-1,rWalk))
b1 = ("better",False,mimic(2,rWalk))
d1 =("deep",False,mimic(8,rWalk))
c1 = ("cheater",False,cheater(repelWalk))
s2 = ("simpleR",False,mimic(4,rWalk))
hr = ("halfRight",False,halfRight(rWalk))
s3 = ("stay",False,mimic(-1,biasWalk(1,0,2)))
sleft =  ("sleft",False,mimic(-1,biasWalk(1,0,0)))
strategies = [s1, s2,b1,b1,d1,s3]
strategies = [s3, s2,sleft,s3]
counts = [0 for x in range(len(strategies))]
for c  in range(1000):
    p, g, m = runGame(strategies)
    counts[p-1] += 1

m = zip([x[0] for x in strategies],counts)
for x in m:
    print(x)

In [ ]:
#| hide
chooseCard(rooms,cardFull)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()